In [1]:
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/drive/MyDrive/740_deeplearning/* /content/
!cp -r /content/drive/MyDrive/740_deeplearning/Novelty/* /content/

Mounted at /content/drive


In [2]:
import math
import torch.nn as nn
from collections import OrderedDict


norm_mean, norm_var = 0.0, 1.0

defaultcfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 512]
relucfg = [2, 6, 9, 13, 16, 19, 23, 26, 29, 33, 36, 39]
convcfg = [0, 3, 7, 10, 14, 17, 20, 24, 27, 30, 34, 37]



class VGG(nn.Module):
    def __init__(self, num_classes=10, init_weights=True, cfg=None, compress_rate=None):
        super(VGG, self).__init__()
        self.features = nn.Sequential()

        if cfg is None:
            cfg = defaultcfg

        self.relucfg = relucfg
        self.covcfg = convcfg
        self.compress_rate = compress_rate
        self.features = self.make_layers(cfg[:-1], True, compress_rate)
        self.classifier = nn.Sequential(OrderedDict([
            ('linear1', nn.Linear(cfg[-2], cfg[-1])),
            ('norm1', nn.BatchNorm1d(cfg[-1])),
            ('relu1', nn.ReLU(inplace=True)),
            ('linear2', nn.Linear(cfg[-1], num_classes)),
        ]))

        if init_weights:
            self._initialize_weights()

    def make_layers(self, cfg, batch_norm=True, compress_rate=None):
        layers = nn.Sequential()
        in_channels = 3
        cnt = 0
        for i, v in enumerate(cfg):
            if v == 'M':
                layers.add_module('pool%d' % i, nn.MaxPool2d(kernel_size=2, stride=2))
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                conv2d.cp_rate = compress_rate[cnt]
                cnt += 1

                layers.add_module('conv%d' % i, conv2d)
                layers.add_module('norm%d' % i, nn.BatchNorm2d(v))
                layers.add_module('relu%d' % i, nn.ReLU(inplace=True))
                in_channels = v

        return layers

    def forward(self, x):
        x = self.features(x)

        x = nn.AvgPool2d(2)(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(0.5)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


def vgg_16_bn(compress_rate=[0.95]+[0.5]*6+[0.9]*4+[0.8]*2):
    return VGG(compress_rate=compress_rate)


In [ ]:
import os
import warnings
import torch
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.transforms as transforms
from autoattack import AutoAttack
from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Load model
def load_model(model_path):
    model = vgg_16_bn()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    ckpt = torch.load(model_path, map_location=device)
    model.load_state_dict(ckpt['state_dict'])
    model.to(device)
    model.eval()
    return model

# Load data
def load_data(data_dir):
    pin_memory = True

    transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])

    testset = CIFAR10(root=data_dir, train=False, download=True, transform=transform_test)
    loader_test = DataLoader(
            testset, batch_size=1024, shuffle=False, 
            num_workers=2, pin_memory=pin_memory)
        
    return loader_test

# Create save directory
def create_save_dir(save_dir):
    os.makedirs(save_dir, exist_ok=True)

# Get attack
def get_attack(model, norm, epsilon, log_path, version):
    adversary = AutoAttack(model, norm=norm, eps=epsilon,
                           log_path=log_path, version=version)
    return adversary

def main():
    warnings.filterwarnings("ignore")

    data_dir = './data'
    norm = 'L2'  # Linf
    epsilon = 8. / 255.
    model_path = '/content/drive/MyDrive/740_deeplearning/Novelty/CRANK_vgg_16_bn_cov12.pt'
    # model_path = '/content/drive/MyDrive/740_deeplearning/Novelty/HRANK_vgg_16_bn_cov12.pt'
    # model_path = '/content/drive/MyDrive/740_deeplearning/Novelty/vgg_16_bn.pt'
    individual = False
    save_dir = f'./content/drive/MyDrive/740_deeplearning/Novelty/results_CRANK/AA_results_cifar10_{norm}'
    # save_dir = f'./content/drive/MyDrive/740_deeplearning/Novelty/results_HRANK/AA_results_cifar10_{norm}'
    # save_dir = f'./content/drive/MyDrive/740_deeplearning/Novelty/results_no_pretrain/AA_results_cifar10_{norm}'
    batch_size = 1024
    version = 'standard'
    state_path = None

    create_save_dir(save_dir)
    log_path = os.path.join(save_dir, 'log_file.txt')

    model = load_model(model_path)
    test_loader = load_data(data_dir)
    n_ex = len(test_loader.dataset)

    # Get data and labels from test_loader
    x_test = torch.cat([x for (x, y) in test_loader], 0)
    y_test = torch.cat([y for (x, y) in test_loader], 0)
    
    adversary = get_attack(model, norm, epsilon, log_path, version)

    with torch.no_grad():
        if not individual:

            adv_complete = adversary.run_standard_evaluation(x_test[:n_ex], y_test[:n_ex],
                                                              bs=batch_size, state_path=state_path)

            torch.save({'adv_complete': adv_complete}, f'{save_dir}/aa_{version}_1_{adv_complete.shape[0]}_eps_{epsilon:.5f}.pth')
            
        else:

            adv_complete = adversary.run_standard_evaluation_individual(x_test[:n_ex],
                                                                        y_test[:n_ex], bs=batch_size)
            
            torch.save(adv_complete, f'{save_dir}/aa_{version}_individual_1_{n_ex}_eps_{epsilon:.5f}.pth')

if __name__ == '__main__':
    main()